In [3]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F

/root/anaconda3/envs/word/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = 'cuda'
batch_size = 32
num_epochs = 100
learning_rate = 0.001

In [5]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 10, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(10, 20, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(20, 32, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(32*9*9, 256, bias=False),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2, stride=2),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 26)
        )
        self.dropout = torch.nn.Dropout(0.3)
    
    def forward(self, x):

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.dropout(x)
        out = self.layer4(x.reshape(x.shape[0], -1)) # input: batch_size x all_features

        return out # batch_size x 26

In [17]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()
    
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=3, stride=2, padding=1)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=3, stride=2, padding=1)
        self.conv3 = torch.nn.Conv2d(20, 32, kernel_size=3, stride=2, padding=1)
        self.linear1 = torch.nn.Linear(32*9*9, 256)
        self.linear2 = torch.nn.Linear(128, 64)
        self.linear3 = torch.nn.Linear(64, 26)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)
        self.max_pool2d = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.max_pool1d = torch.nn.MaxPool1d(kernel_size=2, stride=2)
    
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.max_pool2d(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.max_pool2d(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        x = self.max_pool2d(x)
        print(x.shape)
        x = x.reshape(x.shape[0], -1)
        print(x.shape)
        x = self.linear1(x)
        print(x.shape)
        x = self.max_pool1d(x)
        print(x.shape)
        x = self.linear2(x)
        print(x.shape)
        x = self.linear3(x)
        print(x.shape)
        out = x

        return out # batch_size x 26

In [18]:
model = Classifier().to(device)

In [19]:
data = torch.rand((batch_size, 1, 600, 600)).to(device) # batch x 1 x 600 x 600 (batch, channel, width, height)

In [20]:
pred = model(data)

torch.Size([32, 1, 600, 600])
torch.Size([32, 10, 300, 300])
torch.Size([32, 10, 150, 150])
torch.Size([32, 20, 75, 75])
torch.Size([32, 20, 37, 37])
torch.Size([32, 32, 19, 19])
torch.Size([32, 32, 9, 9])
torch.Size([32, 2592])
torch.Size([32, 256])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 26])


In [9]:
pred[0]

tensor([ 0.0012, -0.0746, -0.0796, -0.0873,  0.0594, -0.1061,  0.0502, -0.0424,
         0.0047,  0.1587,  0.0389, -0.1296, -0.1328,  0.0540,  0.0680,  0.0615,
        -0.0664, -0.0823,  0.0388,  0.0885,  0.0009,  0.0117,  0.0006,  0.0541,
         0.0051, -0.0097], device='cuda:0', grad_fn=<SelectBackward0>)

In [25]:
softmax = torch.nn.Softmax(dim=1)

In [26]:
prob = softmax(pred)

In [30]:
torch.argmax(prob[0])

tensor(9, device='cuda:0')

In [31]:
prob[0]

tensor([0.0386, 0.0358, 0.0356, 0.0353, 0.0409, 0.0347, 0.0405, 0.0369, 0.0387,
        0.0452, 0.0401, 0.0338, 0.0337, 0.0407, 0.0412, 0.0410, 0.0361, 0.0355,
        0.0401, 0.0421, 0.0386, 0.0390, 0.0386, 0.0407, 0.0387, 0.0382],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [39]:
torch.argmax(pred, axis=1)

tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9], device='cuda:0')

In [ ]:
def cal_acc(pred, label):
    '''
    pred: batch_size x 26 with probabilities
    label: batch_size x 26 in one-hot format
    '''
    pred_word = torch.argmax(pred, axis=1)
    label_word = torch.argmax(label, axis=1)
    correct_pred_counts = torch.sum(pred_word == pred_word)
    
    acc = correct_pred_counts.item() / len(pred) # just float not torch
    
    return acc

In [42]:
pred = torch.randn((32,26))
label = torch.randn((32,26))

In [48]:
torch.sum(pred_word == pred_word)

tensor(32)

In [ ]:
pred_word